In [2]:
import numpy as np
import torch
import cv2
import os
import voc12.data
import scipy.misc
import importlib
from torch.utils.data import DataLoader
import torchvision
from tool import imutils, pyutils, visualization
import argparse
from PIL import Image
import torch.nn.functional as F
import pandas as pd

In [3]:
model = getattr(importlib.import_module("network.resnet38_SEAM"), 'Net')()
model.load_state_dict(torch.load("resnet38_SEAM.pth"))
model.eval()
model.cuda()

Net(
  (conv1a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (b2): ResBlock(
    (bn_branch2a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv_branch1): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  )
  (b2_1): ResBlock(
    (bn_branch2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2a): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_branch2b1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_branch2b1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad

In [4]:
infer_dataset = voc12.data.VOC12ClsDatasetMSF("voc12/bb_list.txt", voc12_root="/home/bnair/data/chest14",
                                                  scales=[0.5, 1.0, 1.5, 2.0],
                                                  inter_transform=torchvision.transforms.Compose(
                                                       [np.asarray,
                                                        model.normalize,
                                                        imutils.HWC_to_CHW]))

infer_data_loader = DataLoader(infer_dataset, shuffle=False, num_workers=8, pin_memory=True)

In [5]:
# n_gpus = torch.cuda.device_count()
# model_replicas = torch.nn.parallel.replicate(model, list(range(n_gpus)))

In [6]:
n_gpus = 1
model_replicas = torch.nn.parallel.replicate(model, list(range(n_gpus)))

In [7]:
from torchvision.transforms.functional import to_pil_image
from torchcam.utils import overlay_mask
import torchvision.transforms as T

# get list of images in outcam/ folder
outcam_img_list = os.listdir("outcampred/")
print(outcam_img_list)

for iter, (img_name, img_list, label) in enumerate(infer_data_loader):
    img_name = img_name[0]; label = label[0]

    if img_name + '.png' not in outcam_img_list:
        print("{} not in outcampred/".format(img_name))
        img_path = voc12.data.get_img_path(img_name, "/home/bnair/data/chest14")
        orig_img = np.asarray(Image.open(img_path))
        orig_img_size = orig_img.shape[:2]
        def _work(i, img):
                with torch.no_grad():
                    with torch.cuda.device(i%n_gpus):
                        _, cam = model_replicas[i%n_gpus](img.cuda())
                        cam = F.upsample(cam[:,1:,:,:], orig_img_size, mode='bilinear', align_corners=False)[0]
                        cam = cam.cpu().numpy() * label.clone().view(20, 1, 1).numpy()
                        # print(cam)
                        if i % 2 == 1:
                            cam = np.flip(cam, axis=-1)
                        return cam

        thread_pool = pyutils.BatchThreader(_work, list(enumerate(img_list)),
                                                batch_size=2, prefetch_size=0, processes=1)
        
        cam_list = thread_pool.pop_results()
        sum_cam = np.sum(cam_list, axis=0)
        sum_cam[sum_cam < 0] = 0
        cam_max = np.max(sum_cam, (1,2), keepdims=True)
        cam_min = np.min(sum_cam, (1,2), keepdims=True)
        sum_cam[sum_cam < cam_min+1e-5] = 0
        norm_cam = (sum_cam-cam_min-1e-5) / (cam_max - cam_min + 1e-5)

        cam_dict = {}
        for i in range(20):
            if label[i] > 1e-5:
                cam_dict[i] = norm_cam[i]

        np.save(os.path.join('/home/bnair/SEAM/outcam', img_name + '.npy'), cam_dict)

        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l = lab[:,:,0]
        threshold = cv2.threshold(l, 200, 255, cv2.THRESH_BINARY)[1]
        blur = cv2.GaussianBlur(threshold, (5,5), 0)
        heatmap = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
        result = cv2.addWeighted(img, 0.1, heatmap, 0.5, 0)
        cv2.imwrite('/home/bnair/SEAM/outcampred/' + img_name + '.png', result)
        break

    else:
        print("Already processed: " + img_name)


['0026132_016.png', '0009889_018.png', '0019177_000.png', '0022416_004.png', '0012094_040.png', '0012636_000.png', '0003028_006.png', '0021132_000.png', '0000643_002.png', '0019706_002.png', '0009779_001.png', '0016522_023.png', '0009342_000.png', '0001170_046.png', '0013807_009.png', '0001836_041.png', '0000468_017.png', '0013911_000.png', '0021748_000.png', '0012376_010.png', '0021420_027.png', '0025707_015.png', '0026087_000.png', '0028383_002.png', '0016786_009.png', '0010767_016.png', '0018762_001.png', '0011576_000.png', '0005532_016.png', '0008005_004.png', '0010610_003.png', '0014015_003.png', '0016487_002.png', '0018366_010.png', '0025787_027.png', '0020673_005.png', '0015069_001.png', '0012892_010.png', '0008547_001.png', '0011832_002.png', '0025368_018.png', '0015895_017.png', '0001534_005.png', '0026848_007.png', '0016184_040.png', '0021381_013.png', '0018496_007.png', '0019706_012.png', '0023325_019.png', '0025954_025.png', '0020318_007.png', '0009229_003.png', '0019373_05

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/bnair/SEAM/venv/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/bnair/SEAM/venv/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 289, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
 

KeyboardInterrupt: 

In [8]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [9]:
from chitra.image import Chitra
from PIL import Image
from tqdm import tqdm

bboxes = pd.read_csv('./voc12/BBox_List_2017.csv')
localization_images = {}
for index, row in tqdm(bboxes.iterrows()):
    img_name = row['Image Index']
    if img_name[1:] in os.listdir('/home/bnair/SEAM/outcampred'):
        box = [[row['x'], row['y'], row['h'], row['w']]]
        label = [row['Finding Label']]
        image = Chitra('/home/bnair/data/chest14/'+img_name, labels=label, bboxes=box, box_format='xyhw')
        # image.resize_image_with_bbox((224, 224))
        localization_images[img_name] = {
            'label': row['Finding Label'],
            'xA': (image.bboxes[0].x1/1024)*224,
            'yA': (image.bboxes[0].y1/1024)*224,
            'xB': (image.bboxes[0].x2/1024)*224,
            'yB': (image.bboxes[0].y2/1024)*224,
        }

984it [00:00, 994.00it/s] 


In [10]:
from chitra.image import Chitra

def image_to_bboxes(image_file):
    bboxes = pd.read_csv('./BBox_List_2017.csv')
    img_row = bboxes[bboxes['Image Index'] == image_file]
    return (img_row['x'].values[0]/1024), (img_row['y'].values[0]/1024), (img_row['w'].values[0]/1024), (img_row['h'].values[0]/1024)

In [11]:
def get_class_from_image(image_file):
    bboxes = pd.read_csv('./BBox_List_2017.csv')
    img_row = bboxes[bboxes['Image Index'] == image_file]
    return img_row['Finding Label'].values[0]

In [12]:
scores = {}
diseases_available = bboxes['Finding Label'].unique()
for disease in diseases_available:
    scores[disease] = {
        'count': bboxes[bboxes['Finding Label'] == disease].shape[0],
        'evaluated': 0,
        'total_iou': 0,
        'total_cdice': 0,
    }

In [14]:
# # iterate over images in outcampred folder and draw bounding boxes
# for img_name in os.listdir('/home/bnair/SEAM/outcampred'):
#     img = cv2.imread(os.path.join('/home/bnair/SEAM/outcampred', img_name))
#     img = cv2.resize(img, (224, 224))
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
#     contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     contours = contours[0] if len(contours) == 2 else contours[1]
#     bboxes = []
#     for cnt in contours:
#         x, y, w, h = cv2.boundingRect(cnt)
#         if x == 0 or x == 224 - 1 or y == 0 or y == 224 - 1:
#             continue
#         bboxes.append((x,y,w,h))
#     if len(bboxes) == 0:
#         scores[localization_images["0"+img_name]['label']]['evaluated'] += 1
#         continue
#     ious = []
#     iou_bb = {}
#     gt_XA, gt_YA, gt_XB, gt_YB = 0, 0, 0, 0
#     xA, yA, xB, yB = 0, 0, 0, 0
#     for bbox in bboxes:
#         xA, yA, xB, yB = bbox
#         label = localization_images["0"+img_name]['label']
#         gt_XA = localization_images["0"+img_name]['xA']
#         gt_YA = localization_images["0"+img_name]['yA']
#         gt_XB = localization_images["0"+img_name]['xB']
#         gt_YB = localization_images["0"+img_name]['yB']
#         # draw both the boundiug boxes and the label on the image
#         ious.append(bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)]))
#         iou_bb[bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)])] = bbox
#     # print(ious)
#     scores[label]['total_iou'] += max(ious)
#     scores[label]['bbox'] = iou_bb[max(ious)] 
#     scores[label]['evaluated'] += 1
#     xA, yA, xB, yB = scores[label]['bbox']
#     cv2.rectangle(img, (int(xA), int(yA)), (int(xB), int(yB)), (0, 255, 0), 2)
#     cv2.rectangle(img, (int(gt_XA), int(gt_YA)), (int(gt_XB), int(gt_YB)), (0, 0, 255), 2)
#     cv2.putText(img, label, (xA, yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#     cv2.imwrite(os.path.join('/home/bnair/SEAM/outcamgtbb', label + '_' + img_name), img)

In [15]:
for img_name in os.listdir('/home/bnair/SEAM/outcampred'):
    img = cv2.imread(os.path.join('/home/bnair/SEAM/outcampred', img_name))
    img = cv2.resize(img, (224, 224))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bboxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if x == 0 or x == 224 - 1 or y == 0 or y == 224 - 1:
            continue
        bboxes.append((x,y,w,h))
    if len(bboxes) == 0:
        scores[localization_images["0"+img_name]['label']]['evaluated'] += 1
        continue
    ious = []
    iou_bb = {}
    gt_XA, gt_YA, gt_XB, gt_YB = 0, 0, 0, 0
    xA, yA, xB, yB = 0, 0, 0, 0
    for bbox in bboxes:
        xA, yA, xB, yB = bbox
        label = localization_images["0"+img_name]['label']
        gt_XA = localization_images["0"+img_name]['xA']
        gt_YA = localization_images["0"+img_name]['yA']
        gt_XB = localization_images["0"+img_name]['xB']
        gt_YB = localization_images["0"+img_name]['yB']
        # draw both the boundiug boxes and the label on the image
        ious.append(bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)]))
        iou_bb[bb_intersection_over_union([int(xA), int(yA), int(xB), int(yB)], [int(gt_XA), int(gt_YA), int(gt_XB), int(gt_YB)])] = bbox
    # print(ious)
    scores[label]['total_iou'] += max(ious)
    scores[label]['bbox'] = iou_bb[max(ious)] 
    scores[label]['evaluated'] += 1
    xA, yA, xB, yB = scores[label]['bbox']
    img = cv2.imread(os.path.join('/home/bnair/data/chest14', "0" + img_name))
    img = cv2.resize(img, (224, 224))
    cv2.rectangle(img, (int(xA), int(yA)), (int(xB), int(yB)), (0, 255, 0), 2)
    cv2.rectangle(img, (int(gt_XA), int(gt_YA)), (int(gt_XB), int(gt_YB)), (0, 0, 255), 2)
    cv2.imwrite(os.path.join('/home/bnair/SEAM/outcamgtbb', label + '_' + img_name), img)

0026132_016.png
0009889_018.png
0019177_000.png
0022416_004.png
0012094_040.png
0012636_000.png
0003028_006.png
0021132_000.png
0000643_002.png
0019706_002.png
0009779_001.png
0016522_023.png
0009342_000.png
0001170_046.png
0013807_009.png
0001836_041.png
0000468_017.png
0013911_000.png
0021748_000.png
0012376_010.png
0021420_027.png
0025707_015.png
0026087_000.png
0028383_002.png
0016786_009.png
0010767_016.png
0018762_001.png
0011576_000.png
0005532_016.png
0008005_004.png
0010610_003.png
0014015_003.png
0016487_002.png
0018366_010.png
0025787_027.png
0020673_005.png
0015069_001.png
0012892_010.png
0008547_001.png
0011832_002.png
0025368_018.png
0015895_017.png
0001534_005.png
0026848_007.png
0016184_040.png
0021381_013.png
0018496_007.png
0019706_012.png
0023325_019.png
0025954_025.png
0020318_007.png
0009229_003.png
0019373_058.png
0023176_010.png
0006096_010.png
0003787_003.png
0028518_021.png
0016417_008.png
0013471_002.png
0013337_000.png
0012045_009.png
0001555_002.png
0017403_

In [13]:
scores

{'Atelectasis': {'count': 180,
  'evaluated': 159,
  'total_iou': 4.701688452741005,
  'total_cdice': 0},
 'Cardiomegaly': {'count': 146,
  'evaluated': 135,
  'total_iou': 1.5517014024967464,
  'total_cdice': 0},
 'Effusion': {'count': 153,
  'evaluated': 115,
  'total_iou': 2.987950502612605,
  'total_cdice': 0},
 'Infiltrate': {'count': 123,
  'evaluated': 101,
  'total_iou': 1.3577550234457003,
  'total_cdice': 0},
 'Mass': {'count': 85,
  'evaluated': 78,
  'total_iou': 0.6007425894444609,
  'total_cdice': 0},
 'Nodule': {'count': 79,
  'evaluated': 76,
  'total_iou': 3.49469406302026,
  'total_cdice': 0},
 'Pneumonia': {'count': 120,
  'evaluated': 118,
  'total_iou': 2.8736827058251113,
  'total_cdice': 0},
 'Pneumothorax': {'count': 98,
  'evaluated': 98,
  'total_iou': 0.7602607666649264,
  'total_cdice': 0}}

In [14]:
# find average iou for each disease
for disease in scores:
    scores[disease]['avg_iou'] = scores[disease]['total_iou'] / scores[disease]['evaluated']

In [15]:
scores

{'Atelectasis': {'count': 180,
  'evaluated': 159,
  'total_iou': 4.701688452741005,
  'total_cdice': 0,
  'avg_iou': 0.029570367627301922},
 'Cardiomegaly': {'count': 146,
  'evaluated': 135,
  'total_iou': 1.5517014024967464,
  'total_cdice': 0,
  'avg_iou': 0.011494084462938862},
 'Effusion': {'count': 153,
  'evaluated': 115,
  'total_iou': 2.987950502612605,
  'total_cdice': 0,
  'avg_iou': 0.02598217828358787},
 'Infiltrate': {'count': 123,
  'evaluated': 101,
  'total_iou': 1.3577550234457003,
  'total_cdice': 0,
  'avg_iou': 0.013443119044016835},
 'Mass': {'count': 85,
  'evaluated': 78,
  'total_iou': 0.6007425894444609,
  'total_cdice': 0,
  'avg_iou': 0.007701828069800781},
 'Nodule': {'count': 79,
  'evaluated': 76,
  'total_iou': 3.49469406302026,
  'total_cdice': 0,
  'avg_iou': 0.04598281661868763},
 'Pneumonia': {'count': 120,
  'evaluated': 118,
  'total_iou': 2.8736827058251113,
  'total_cdice': 0,
  'avg_iou': 0.024353243269704332},
 'Pneumothorax': {'count': 98,
  